In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [4]:
len(words)

32033

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i for i, s in enumerate(chars, start=1)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [93]:
block_size = 3
X, Y = [], []

for w in words[:100]:
    # print(w)

    context = [0] * block_size

    for ch in w+'.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [94]:
X.shape

torch.Size([684, 3])

In [39]:
Y

tensor([ 1,  0,  2,  0,  9,  0,  7,  0,  1,  0,  9,  0, 12,  0])

In [40]:
C = torch.randn((27, 2))
C

tensor([[ 0.1760,  1.0203],
        [-0.2992, -0.3132],
        [ 0.2024, -0.3995],
        [ 0.0201, -1.1162],
        [-0.3158,  1.1883],
        [-0.4174, -0.2063],
        [ 0.4403,  0.7183],
        [ 1.0431, -0.3735],
        [ 1.5691, -0.4557],
        [-0.4270, -0.4444],
        [-0.3078,  0.0131],
        [ 1.7649, -0.1727],
        [-0.6510, -1.1650],
        [-0.0055, -0.4970],
        [-0.6470, -0.3853],
        [-1.4654,  1.3320],
        [-0.6485,  1.3985],
        [-0.0037,  0.1286],
        [ 1.5749,  0.2032],
        [-0.5157,  0.5502],
        [-0.6774,  1.1963],
        [ 0.0034, -1.3533],
        [-0.0033, -0.5547],
        [ 0.2018,  1.1322],
        [-1.1667, -0.3311],
        [ 0.1518, -0.2643],
        [-0.3373, -1.5741]])

In [41]:
C[5]

tensor([-0.4174, -0.2063])

In [42]:
F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-0.4174, -0.2063])

In [43]:
emb = C[X]
emb.shape

torch.Size([14, 3, 2])

In [44]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [45]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [46]:
h.shape

torch.Size([14, 100])

In [47]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [48]:
logits = h @ W2 + b2

In [49]:
logits.shape

torch.Size([14, 27])

In [50]:
counts = logits.exp()

In [51]:
probs = counts / counts.sum(1, keepdim=True)

In [52]:
probs.shape

torch.Size([14, 27])

In [55]:
loss = -probs[torch.arange(14), Y].log().mean()
loss

tensor(17.1978)

In [56]:
# now made respectable

In [95]:
g = torch.Generator().manual_seed(42)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.rand((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [96]:
sum(p.nelement() for p in parameters)

3481

In [97]:
for p in parameters:
    p.requires_grad = True

In [125]:
for _ in range(10):
    # forward pass
    emb = C[X] # (n, 3, 2)
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2 # (n, 27)
    # counts = logits.exp()
    # probs = counts / counts.sum(1, keepdim=True)
    # loss = -probs[torch.arange(probs.shape[0]), Y].log().mean()
    loss = F.cross_entropy(logits, Y)
    print(f'{loss=}')
    # backward
    for p in parameters:
        p.grad = None

    loss.backward()
    # update
    for p in parameters:
        p.data += -0.5 * p.grad

loss=tensor(1.1811, grad_fn=<NllLossBackward0>)
loss=tensor(1.1844, grad_fn=<NllLossBackward0>)
loss=tensor(1.1793, grad_fn=<NllLossBackward0>)
loss=tensor(1.1826, grad_fn=<NllLossBackward0>)
loss=tensor(1.1776, grad_fn=<NllLossBackward0>)
loss=tensor(1.1808, grad_fn=<NllLossBackward0>)
loss=tensor(1.1758, grad_fn=<NllLossBackward0>)
loss=tensor(1.1791, grad_fn=<NllLossBackward0>)
loss=tensor(1.1741, grad_fn=<NllLossBackward0>)
loss=tensor(1.1773, grad_fn=<NllLossBackward0>)


In [99]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        ...,
        [ 5, 12,  9],
        [12,  9, 14],
        [ 9, 14,  5]])

In [124]:
pred = []
ix = 0

for _ in range(10):
    p = torch.softmax(logits[ix], dim=0)
    ix = torch.multinomial(p, num_samples=1, replacement=True).item()
    print(itos[ix])
    pred.append(itos[ix])

''.join(pred)

s
b
i
a
l
d
.
a
l
e


'sbiald.ale'